In [1]:
#!pip install web3

In [2]:
#!pip install multicall

In [3]:
import json
import pandas as pd
from web3 import Web3
from multicall import Call, Multicall


def chunks(lst, n):
    """Yield successive n-sized chunks from lst."""
    for i in range(0, len(lst), n):
        yield lst[i:i + n]
        
GearboxAddressProvider = Web3.toChecksumAddress('0xcF64698AFF7E5f27A11dff868AF228653ba53be0')
yVaultUSDC = Web3.toChecksumAddress('0xa354F35829Ae975e850e23e9615b11Da1B3dC4DE')

# The dafault RPC from ethersjs, change it if it doesn't work: https://infura.io/docs
RPC_Endpoint = 'https://mainnet.infura.io/v3/84842078b09946638c03157f83405213'

ABI = """[{"name":"getAccountFactory",
           "inputs":[],
           "outputs":[{"internalType":"address","name":"","type":"address"}],
           "stateMutability":"view","type":"function"},
          {"name":"countCreditAccounts",
           "inputs":[],
           "outputs":[{"internalType":"uint256","name":"","type":"uint256"}],
           "stateMutability":"view","type":"function"},
          {"name":"decimals",
           "inputs":[],
           "outputs":[{"name":"","type":"uint256"}],
           "stateMutability":"view","type":"function"
           }
          ]
      """


w3_eth = Web3(Web3.HTTPProvider(RPC_Endpoint, request_kwargs={'timeout': 20}))
print ('Ethereum connected:', w3_eth.isConnected())

Ethereum connected: True


In [4]:
AccountFactory = w3_eth.eth.contract(address=GearboxAddressProvider, abi=ABI).functions.getAccountFactory().call()
print('AccountFactory:', AccountFactory)

countCreditAccounts = w3_eth.eth.contract(address=AccountFactory, abi=ABI).functions.countCreditAccounts().call()
print('countCreditAccounts:', countCreditAccounts)

yVaultUSDC_decimals = w3_eth.eth.contract(address=yVaultUSDC, abi=ABI).functions.decimals().call()
print('yVaultUSDC_decimals:', yVaultUSDC_decimals)


AccountFactory: 0x444CD42BaEdDEB707eeD823f7177b9ABcC779C04
countCreditAccounts: 5001
yVaultUSDC_decimals: 6


In [5]:
idList = list(range(countCreditAccounts))
multi_idCA = {}

for ids in list(chunks(idList, 400)): #chunk size for multicall = 400
    #d_ca = get_data_multicall(df.loc[id_range], 'creditAccounts', df_abi, AccountFactory)
    multi_result = Multicall([
                    Call(AccountFactory, ['creditAccounts(uint256)(address)', x], [[x, Web3.toChecksumAddress]]) for x in ids
                    ]
                    ,_w3 = w3_eth)
    
    multi_result = multi_result()
    multi_idCA.update(multi_result)
    

In [6]:
multi_idCA

{0: '0xb5DE854F7Db3164c8F9eFeFFED4c06317BCdbBF1',
 1: '0xA3F1E5d5fb80B3bB0F1b04819F0930C4E0f32AF1',
 2: '0xe541B88d68602E5f3Fb511633bAc708BACD8EA4c',
 3: '0xC581Ff1a21f42B315DEDbE219a7Ed2B0fA47aBd5',
 4: '0x5Ce9C22aC551d3b72136B3fe446902B1af0f3654',
 5: '0x149db982DFB72bcF80bdd5cfb86343dec85c2957',
 6: '0xe492413FfADeD383f9fAEb5da229e302F14cBb54',
 7: '0x796384Bd0566B43163c2f5094FD54Ed9B3dA1Ef1',
 8: '0xF91027C04807c71c7771a2314A622dFA0fE509A1',
 9: '0x716C3B303D02F7A0958fF8bC5701039d9876bCE5',
 10: '0x9C612Ca218fB8F40Fc0369BD59E11a4e279C5339',
 11: '0x0B9851B7bcE408dd36d3473a0A0eE8F14ce64ca3',
 12: '0x561cf71ff9d7CfBFB3C867e51583090f69e4EFd0',
 13: '0xe1036ef5cCe675c476D71a5cA74EddAA719b708D',
 14: '0x654C0aDcA189042B16A849d2fF2ED476F525c39e',
 15: '0x01782032De554e86badFC84aB877e5f0eB18dC92',
 16: '0x6443B1370D50dFFd3fe83f28D8a073D8Ac17A17A',
 17: '0x9727545DA1F0AD1FAa811D7eb5bf8d7e71556de4',
 18: '0xC093068230644B46a0c150AF8C22BF052E8F9AB1',
 19: '0xfeDdd9824CD3551c5DB8c24fa1b223E97

In [7]:
multi_idBalance = {}
for ids in list(chunks(list(multi_idCA), 400)): #chunk size for multicall = 400
    multi_result = Multicall([
                    Call(yVaultUSDC, ['balanceOf(address)(uint256)', multi_idCA[x]], [[x, None]]) for x in ids
                    ]
                    ,_w3 = w3_eth)
    
    multi_result = multi_result()
    multi_idBalance.update(multi_result)


In [8]:
multi_idBalance

{0: 0,
 1: 0,
 2: 0,
 3: 0,
 4: 0,
 5: 0,
 6: 0,
 7: 0,
 8: 0,
 9: 0,
 10: 0,
 11: 0,
 12: 0,
 13: 0,
 14: 0,
 15: 0,
 16: 0,
 17: 0,
 18: 0,
 19: 0,
 20: 0,
 21: 0,
 22: 0,
 23: 0,
 24: 0,
 25: 0,
 26: 0,
 27: 0,
 28: 0,
 29: 0,
 30: 0,
 31: 0,
 32: 0,
 33: 0,
 34: 0,
 35: 36823268679,
 36: 0,
 37: 1,
 38: 0,
 39: 0,
 40: 0,
 41: 0,
 42: 0,
 43: 0,
 44: 0,
 45: 0,
 46: 0,
 47: 0,
 48: 0,
 49: 0,
 50: 0,
 51: 0,
 52: 0,
 53: 0,
 54: 0,
 55: 39799123570,
 56: 3952079976,
 57: 0,
 58: 0,
 59: 0,
 60: 0,
 61: 0,
 62: 0,
 63: 0,
 64: 0,
 65: 0,
 66: 0,
 67: 0,
 68: 0,
 69: 0,
 70: 0,
 71: 0,
 72: 0,
 73: 0,
 74: 0,
 75: 0,
 76: 0,
 77: 0,
 78: 0,
 79: 0,
 80: 0,
 81: 0,
 82: 0,
 83: 0,
 84: 0,
 85: 0,
 86: 0,
 87: 0,
 88: 0,
 89: 0,
 90: 0,
 91: 0,
 92: 0,
 93: 0,
 94: 0,
 95: 0,
 96: 0,
 97: 0,
 98: 0,
 99: 0,
 100: 0,
 101: 0,
 102: 0,
 103: 0,
 104: 0,
 105: 0,
 106: 0,
 107: 39776257801,
 108: 0,
 109: 0,
 110: 0,
 111: 0,
 112: 0,
 113: 0,
 114: 0,
 115: 0,
 116: 0,
 117: 0,
 118: 0,


In [9]:
multi_CA = {multi_idCA[x]:multi_idBalance[x] for x in multi_idCA}
multi_CA

{'0xb5DE854F7Db3164c8F9eFeFFED4c06317BCdbBF1': 0,
 '0xA3F1E5d5fb80B3bB0F1b04819F0930C4E0f32AF1': 0,
 '0xe541B88d68602E5f3Fb511633bAc708BACD8EA4c': 0,
 '0xC581Ff1a21f42B315DEDbE219a7Ed2B0fA47aBd5': 0,
 '0x5Ce9C22aC551d3b72136B3fe446902B1af0f3654': 0,
 '0x149db982DFB72bcF80bdd5cfb86343dec85c2957': 0,
 '0xe492413FfADeD383f9fAEb5da229e302F14cBb54': 0,
 '0x796384Bd0566B43163c2f5094FD54Ed9B3dA1Ef1': 0,
 '0xF91027C04807c71c7771a2314A622dFA0fE509A1': 0,
 '0x716C3B303D02F7A0958fF8bC5701039d9876bCE5': 0,
 '0x9C612Ca218fB8F40Fc0369BD59E11a4e279C5339': 0,
 '0x0B9851B7bcE408dd36d3473a0A0eE8F14ce64ca3': 0,
 '0x561cf71ff9d7CfBFB3C867e51583090f69e4EFd0': 0,
 '0xe1036ef5cCe675c476D71a5cA74EddAA719b708D': 0,
 '0x654C0aDcA189042B16A849d2fF2ED476F525c39e': 0,
 '0x01782032De554e86badFC84aB877e5f0eB18dC92': 0,
 '0x6443B1370D50dFFd3fe83f28D8a073D8Ac17A17A': 0,
 '0x9727545DA1F0AD1FAa811D7eb5bf8d7e71556de4': 0,
 '0xC093068230644B46a0c150AF8C22BF052E8F9AB1': 0,
 '0xfeDdd9824CD3551c5DB8c24fa1b223E97fb9258B': 0,


In [10]:
#only with existing balance
{key:value for key, value in multi_CA.items() if value > 0}

{'0xfBA6b8817699cc4B90983887F3B96579f366bd5a': 36823268679,
 '0x9b7249E6D6fdd05295d8E5ef68a1068dec1A6d07': 1,
 '0x7a2f98848C626815d93957c1D2637fCE7cCa9fC0': 39799123570,
 '0x8e996336FEe5CE8F6BC3AfD68e5b40f6Cef8cb87': 3952079976,
 '0xfD96Bc2034061e0E02bbB534B4c99C0FED30160D': 39776257801,
 '0xE5AAe8E98607fE92C04e09AD023bcF9dD91999C5': 39655792262,
 '0x9dE7BE4057872Af76a6C3eae5FF1219cce9adAB9': 40691007865,
 '0x60c230b831eeF255Dad4d5244170E98B8F361C21': 3969974252,
 '0xd7FD97d43f790F87bc62A3B3979Df20DE6DDEB2A': 128970198740,
 '0x16cF8A14a0B20e4150F25D7b8e42b653Ef5bca30': 39332161711,
 '0xAB2d65eA191313AEFCD0C48A6f7D6b20852bAE0C': 40785830856,
 '0xaBBd655b3791175113c1f1146D3B369494A2b815': 39639734244,
 '0x5dBCFABBa5482206FfaE8446ae00ce22CA767e46': 19302346058,
 '0x9554E3C31Aa1e2cd43C79C7Fed4d166B39a1aaf6': 39554918657,
 '0x1F95B6AC9BaCcCAF7121152b704D6b3820A81f32': 16037508788}

In [11]:
#decimal values
{key:value*10**(-yVaultUSDC_decimals) for key, value in multi_CA.items() if value > 0}

{'0xfBA6b8817699cc4B90983887F3B96579f366bd5a': 36823.268679,
 '0x9b7249E6D6fdd05295d8E5ef68a1068dec1A6d07': 1e-06,
 '0x7a2f98848C626815d93957c1D2637fCE7cCa9fC0': 39799.123569999996,
 '0x8e996336FEe5CE8F6BC3AfD68e5b40f6Cef8cb87': 3952.079976,
 '0xfD96Bc2034061e0E02bbB534B4c99C0FED30160D': 39776.257801,
 '0xE5AAe8E98607fE92C04e09AD023bcF9dD91999C5': 39655.792261999995,
 '0x9dE7BE4057872Af76a6C3eae5FF1219cce9adAB9': 40691.007865,
 '0x60c230b831eeF255Dad4d5244170E98B8F361C21': 3969.974252,
 '0xd7FD97d43f790F87bc62A3B3979Df20DE6DDEB2A': 128970.19873999999,
 '0x16cF8A14a0B20e4150F25D7b8e42b653Ef5bca30': 39332.161711,
 '0xAB2d65eA191313AEFCD0C48A6f7D6b20852bAE0C': 40785.830856,
 '0xaBBd655b3791175113c1f1146D3B369494A2b815': 39639.734244,
 '0x5dBCFABBa5482206FfaE8446ae00ce22CA767e46': 19302.346058,
 '0x9554E3C31Aa1e2cd43C79C7Fed4d166B39a1aaf6': 39554.918656999995,
 '0x1F95B6AC9BaCcCAF7121152b704D6b3820A81f32': 16037.508788}